In [ ]:
# LAB | Connecting Python to SQL

In [ ]:
pip install sqlalchemy pymysql pandas

In [7]:
from sqlalchemy import create_engine
import pandas as pd

# Reemplaza con tus credenciales de base de datos
db_user = 'root'         
db_password = 'password'  
db_host = 'localhost'         
db_port = '3306'              
db_name = 'sakila'          

# Crear el motor de la base de datos
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Probar la conexión
try:
    with engine.connect() as connection:
        result = pd.read_sql("SELECT DATABASE();", connection)
        print(f"Conectado a: {result.iloc[0, 0]}")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")

Conectado a: sakila


In [8]:
# 2

def rentals_month(engine, month, year):
    query = f"""
    SELECT rental_id, rental_date, inventory_id, customer_id, return_date, staff_id
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df

# Ejemplo de uso:
may_rentals = rentals_month(engine, 5, 2005)
print(may_rentals.head())


   rental_id         rental_date  inventory_id  customer_id  \
0          1 2005-05-24 22:53:30           367          130   
1          2 2005-05-24 22:54:33          1525          459   
2          3 2005-05-24 23:03:39          1711          408   
3          4 2005-05-24 23:04:41          2452          333   
4          5 2005-05-24 23:05:21          2079          222   

          return_date  staff_id  
0 2005-05-26 22:04:30         1  
1 2005-05-28 19:40:33         1  
2 2005-06-01 22:12:39         1  
3 2005-06-03 01:43:41         2  
4 2005-06-02 04:33:21         1  


In [9]:
# 3

def rental_count_month(df, month, year):
    rental_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_count

# Ejemplo de uso:
may_rental_count = rental_count_month(may_rentals, 5, 2005)
print(may_rental_count.head())


   customer_id  rentals_05_2005
0            1                2
1            2                1
2            3                2
3            5                3
4            6                3


In [10]:
# 4

def compare_rentals(df1, df2, month1, year1, month2, year2):
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    merged_df['difference'] = merged_df[f'rentals_{month2:02d}_{year2}'] - merged_df[f'rentals_{month1:02d}_{year1}']
    return merged_df

# Ejemplo de uso:
june_rentals = rentals_month(engine, 6, 2005)
june_rental_count = rental_count_month(june_rentals, 6, 2005)

comparison = compare_rentals(may_rental_count, june_rental_count, 5, 2005, 6, 2005)
print(comparison.head())


   customer_id  rentals_05_2005  rentals_06_2005  difference
0            1              2.0              7.0         5.0
1            2              1.0              1.0         0.0
2            3              2.0              4.0         2.0
3            5              3.0              5.0         2.0
4            6              3.0              4.0         1.0
